# Script to produce CCA S2S forecast and assess associated skill, using NCEP subseasonal rainfall, trained vs IMD data.

Authors: Andrew W. Robertson (awr@iri.columbia.edu), Á.G. Muñoz, and Remi Cousin

This version: 18 Jul 2018, Modified by AWR

First version: 12 Dec 2017; First IPython version (Jupyter Notebook), 24 Jun 2018

### Output:
* Several skill maps for assessment of deterministic forecast, in the output folder.
* CPT scripts used to assess skill, in the scripts folder.
* Downloaded input files, in the input folder.

### Notes:
1. Old data in the input folder is deleted at the beginning of the process!
2. *Weekly* initializations available per month are used, concatenated.
3. The T coordinate has been faked, so CPT can deal with all the initializations.
4. Rainfall observations are IMD data (India) at 1 deg. (Land only)
5. IMD Lead week DEFINITIONS: week 1 3-9; week 2 10-16; week 3+4 17-30:
6. Training: Uses the calendar month containing the forecast start day (not optimal!)
7. Grads output: if desired (still needs to bde made user-selectable in [1]
8. Multi-lead loop version: runs week1, week2, week3-4 sequentially 
9. climo is defined as the mean of 3 perturbed runs + control (v2 of python script / v7 bash script)
10. REAL TIME FORECAST USES the 4 6-hourly forecasts from 'fday' only; NB: fday will be YESTERDAY's date in real time (16 members) (v2 of python script / v7 bash script)
11. Automatic plotting of skill scores & forecasts using grads, called from python (v3 of python script)
12. v4: Trains on Jun-Aug season (as opposed to a single month in v3)
13. v5: Rainfall frequency (Only the OBS URL and output files are modified) 3mm is chosen as the threshold here
14. v6: Tunable lagged-ensemble & hindcast stepping
15. v6precip: Uses total weekly rainfall as predictand, in placde of rainfall frequency
16. Renamed to "BiharCFS_S2S" with grads + rfreq switches added for archiving in GitHub (17-Jul-2018)
17. Added prediction limits CPT output& tidied up a bit (18-Jul-2018)

User-modifiable section
----------------------

In [1]:
# Forecast date
mon='Jul' 	# Forecast month 
fyr=2018 	# Forecast year
fday=17 	# Forecast day  (Yesterday in real time)
training_season='Jun-Aug'

hstep = 3 # use all starts in the trainng period with this daily step between them (v5 used 7)
nlag = 3  # length of the lagged ensemble in days
ntrain = 371  # Length of training period 

# Rainfall frequency switch (False gives total rainfall for forecast period)
rainfall_frequency = True
# Wet day threshold (mm)
wetday_threshold = 3
if rainfall_frequency:
    print('Predictand is Rainfall Frequency; wet day threshold = '+str(wetday_threshold))
else:
    print('Predictand is Rainfall Total')

# GrADS plotting switches True/False
# If first switch is False, output will be in CPTv10 format  
grads_plot = False
grads_plot_forecasts = False
grads_plot_skill = False
# Desired skill score, choose from: Spearman, 2AFC, RocAbove, RocBelow
skill_score = 'Spearman'

# Spatial domain for predictor
nla1=32 	# Northernmost latitude
sla1=12 	# Southernmost latitude
wlo1=74 	# Westernmost longitude
elo1=92 	# Easternmost longitude
# Spatial domain for predictand
nla2=27 	# Northernmost latitude
sla2=22 	# Southernmost latitude
wlo2=80 	# Westernmost longitude
elo2=89 	# Easternmost longitude

# Working directory
workdir = '/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2'

# PATH to CPT root directory
cptdir='/Users/andy/Dropbox/pgm/stats/CPT/CPT15/15.7.3/'

# S2S Database key
# AWR key
key='f7a5c7a57103d4ed1be1224f814b01b77951a6c20a4cbe85dd6b1942af7e479b11e3443a819d6bd601ac35070c7e5667e84e23f5'


Predictand is Rainfall Frequency; wet day threshold = 3


Set parameters for leads/target weeks & prepare folders
----------------------------------------------------

In [2]:
import os

# Forecast lead interval
# Lists for looping over lead times
wk = [1,2,34]  # week-lead number label (week1, week2, week3-4)
day1 = [3,10,17]  # first lead day of target weeks 
day2 = [9,16,30]  # last lead day of target weeks 

print("Creating working folders, if not already there...")
%cd $workdir
!mkdir -p input
!mkdir -p output
!mkdir -p scripts
!rm -Rf input/model_*.tsv input/obs_*.tsv  #comment if deletion of old input files is not desired.
!rm -Rf scripts/*

# Set up some parameters
os.system('export CPT_BIN_DIR='+cptdir)
#! export CPT_BIN_DIR=${cptdir}

# Naming of output files
if rainfall_frequency:
    fprefix = 'RFREQ'
else:
    fprefix = 'PRCP'

Creating working folders, if not already there...
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2


Set up Functions
---------------
- these were created using 
%load_ext cell2function
- then put %%cell2function at the top to generate the function

In [3]:
def GetHindcasts(wlo1, elo1, sla1, nla1, day1, day2, mon, os, key, week):  
    # Download hindcasts (NCEP) 
    # v3 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+mon+')/VALUES/S/7/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v5 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/7/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/2068/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag/M%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/'+str(hstep)+'/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    print("\n Hindcasts URL: \n\n "+url)
    os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > model_precip_"+mon+"_wk"+str(week)+".tsv")
    #! curl -g -k -b '__dlauth_id='$key'' ''$url'' > model_precip_${mo}.tsv
    return url
def GetObs(day1, day2, mon, nday, key, week):
    # Download IMD observations  
    # v3 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+mon+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v4 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/2068/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v5 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/3./flagge/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/2068/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v6 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/'+str(hstep)+'/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/3./flagge/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv' 
    # v6 precip: (just omits 3./flagge/T)
    url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/'+str(hstep)+'/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    
    print("\n Obs data URL: \n\n "+url)
    os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > obs_precip_"+mon+"_wk"+str(week)+".tsv")
    #curl -g -k -b '__dlauth_id='$key'' ''$url'' > obs_precip_${mo}.tsv
    return url
def GetObs_RFREQ(day1, day2, mon, nday, key, week, wetday_threshold):
    # Download IMD observations  
    # v3 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+mon+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v4 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/2068/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v5 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/3./flagge/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/2068/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    # v6 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/'+str(hstep)+'/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/3./flagge/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv' 
    # v6 precip: (just omits 3./flagge/T)
    url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+training_season+')/VALUES/S/'+str(hstep)+'/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/'+str(wetday_threshold)+'/flagge/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    
    print("\n Obs data URL: \n\n "+url)
    os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > obs_RFREQ_"+mon+"_wk"+str(week)+".tsv")
    #curl -g -k -b '__dlauth_id='$key'' ''$url'' > obs_precip_${mo}.tsv
    return url
def GetForecast(day1, day2, fday, mon, fyr, nday, wlo1, elo1, sla1, nla1, key, week):
    # Download forecast file  
    # v5 url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/add/4./div/X/70/100/RANGE/Y/0/40/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/L1/removeGRID/S/(0000%20'+str(fday)+'%20'+mon+')/VALUES/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div//mm/unitconvert/SOURCES/.NOAA/.NCEP/.EMC/.CFSv2/.6_hourly_rotating/.FLXF/.surface/.PRATE/S/(0000%20'+str(fday)+'%20'+mon+'%20'+str(fyr)+')/(1800%20'+str(fday)+'%20'+mon+'%20'+str(fyr)+')/RANGE/%5BM%5Daverage/%5BL%5D1/0.0/boxAverage/%5BX/Y%5DregridLinear/L/'+str(day1)+'/'+str(day2)+'/RANGEEDGES/%5BL%5Daverage/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div/(mm/day)/unitconvert/'+str(nday)+'/mul//units/(mm)/def/exch/sub/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/1/Jan/2001/ensotime/12.0/1/Jan/2001/ensotime/%3Agrid/addGRID/T//pointwidth/0/def/pop//name/(tp)/def//units/(mm)/def//long_name/(precipitation_amount)/def/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    #url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag/M%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/L1/removeGRID/S/(0000%20'+str(fday)+'%20'+mon+')/VALUES/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div//mm/unitconvert/SOURCES/.NOAA/.NCEP/.EMC/.CFSv2/.6_hourly_rotating/.FLXF/.surface/.PRATE/%5BL%5D1/0.0/boxAverage/S/-'+str(nlag-1)+'/1/0/shiftdatashort/S/(0000%20'+str(fday)+'%20'+mon+'%20'+str(fyr)+')VALUE/%5BX/Y%5DregridLinear/L/'+str(day1)+'/'+str(day2)+'/RANGEEDGES/%5BL%5Daverage/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div/(mm/day)/unitconvert/'+str(nday)+'/mul//units/(mm)/def/exch/sub/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/1/Jan/2001/ensotime/12.0/1/Jan/2001/ensotime/%3Agrid/addGRID/T//pointwidth/0/def/pop//name/(tp)/def//units/(mm)/def//long_name/(precipitation_amount)/def/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag/M%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag%5Daverage/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/L1/removeGRID/S/(0000%20'+str(fday)+'%20'+mon+')/VALUES/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div//mm/unitconvert/SOURCES/.NOAA/.NCEP/.EMC/.CFSv2/.6_hourly_rotating/.FLXF/.surface/.PRATE/%5BL%5D1/0.0/boxAverage/S/-'+str(nlag-1)+'/1/0/shiftdatashort/%5BS_lag/M%5Daverage/S/(0000%20'+str(fday)+'%20'+mon+'%20'+str(fyr)+')VALUE/%5BX/Y%5DregridLinear/L/'+str(day1)+'/'+str(day2)+'/RANGEEDGES/%5BL%5Daverage/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div/(mm/day)/unitconvert/'+str(nday)+'/mul//units/(mm)/def/exch/sub/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/1/Jan/3001/ensotime/12.0/1/Jan/3001/ensotime/%3Agrid/addGRID/T//pointwidth/0/def/pop//name/(tp)/def//units/(mm)/def//long_name/(precipitation_amount)/def/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
    print("\n Forecast URL: \n\n "+url)
    os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > modelfcst_precip_"+mon+"_fday"+str(fday)+"_wk"+str(week)+".tsv")
    #curl -g -k -b '__dlauth_id='$key'' ''$url'' > modelfcst_precip_fday${fday}.tsv
    return url

Main Loop over leads
-------------------

In [4]:
for L in range(3):
   nday=day2[L]-day1[L]+1	# Length of target period (days) 
   %cd $workdir/input
   GetHindcasts(wlo1, elo1, sla1, nla1, day1[L], day2[L], mon, os, key, wk[L])
   print('Done getting hindcasts, week '+str(wk[L]))
   if rainfall_frequency: 
        GetObs_RFREQ(day1[L], day2[L], mon, nday, key, wk[L], wetday_threshold)
        print('Done getting obs - rfreq')
   else:
        GetObs(day1[L], day2[L], mon, nday, key, wk[L])
        print('Done getting obs - precip')
   GetForecast(day1[L], day2[L], fday, mon, fyr, nday, wlo1, elo1, sla1, nla1, key, wk[L])
   print('Done getting forecasts for week '+str(wk[L]))
   %cd $workdir/scripts

# Set up CPT parameter file
   ! rm -r params
# Opens CCA
   %store 611 > params  
# Opens X input file
   %store 1 >> params
   file='../input/model_precip_'+mon+'_wk'+str(wk[L])+'.tsv'
   %store file >> params
# Nothernmost latitude
   %store nla1 >> params
# Southernmost latitude
   %store sla1 >> params
# Westernmost longitude
   %store wlo1 >> params
 # Easternmost longitude
   %store elo1 >> params

# Minimum number of modes
   %store 1 >> params
# Maximum number of modes
   %store 10 >> params
# Opens forecast (X) file
   %store 3 >> params
   file='../input/modelfcst_precip_'+mon+'_fday'+str(fday)+'_wk'+str(wk[L])+'.tsv'
   %store file >> params

# Opens Y input file
   %store 2 >> params
   if rainfall_frequency:
     file='../input/obs_RFREQ_'+mon+'_wk'+str(wk[L])+'.tsv'
   else:
     file='../input/obs_precip_'+mon+'_wk'+str(wk[L])+'.tsv'  
   %store file >> params
# Northernmost latitude
   %store nla2 >> params
# Southernmost latitude
   %store sla2 >> params
# Westernmost longitude
   %store wlo2 >> params
# Easternmost longitude
   %store elo2 >> params

# Minimum number of modes
   %store 1 >> params
# Maximum number of modes
   %store 10 >> params
# Minimum number of CCA modes
   %store 1 >> params
# Maximum number of CCAmodes
   %store 5 >> params

# X training period
   %store 4 >> params
# First year of X training period
   %store 1901 >> params
# Y training period
   %store 5 >> params
# First year of Y training period
   %store 1901 >> params

# Goodness index
   %store 531 >> params
# Kendalls tau
   %store 3 >> params

# Option: Length of training period
   %store 7 >> params
# Length of training period 
   % store ntrain >> params
#   %store 55 >> params
# Option: Length of cross-validation window
   %store 8 >> params
# Enter length
   %store 3 >> params

# Turn ON Transform predictand data
   %store 541 >> params
# Turn ON zero bound for Y data
#%store 542 >> params
# Turn ON synchronous predictors
   %store 545 >> params
# Turn ON p-values for skill maps
#%store 561 >> params

# Missing value options
   %store 544 >> params
# Missing value X flag:
   blurb='-999'
   %store blurb >> params
# Maximum % of missing values
   %store 10 >> params
# Maximum % of missing gridpoints
   %store 10 >> params
# Number of near-neighbors
   %store 1 >> params
# Missing value replacement : best-near-neighbors
   %store 4 >> params
# Y missing value flag
   blurb='-999'
   %store blurb >> params
# Maximum % of missing values
   %store 10 >> params
# Maximum % of missing stations
   %store 10 >> params
# Number of near-neighbors
   %store 1 >> params
# Best near neighbor
   %store 4 >> params

#554 # Transformation seetings
#1   #Empirical distribution

#######BUILD MODEL AND VALIDATE IT  !!!!!

   if grads_plot:
# select output format
     %store 131 >> params
# GrADS format
     %store 3 >> params
# NB: Default output format is CPTv10 format

# save goodness index
   %store 112 >> params
   file='../output/'+fprefix+'_Kendallstau_'+training_season+'_wk'+str(wk[L])
   %store file >> params

# Cross-validation
   %store 311 >> params

# cross-validated skill maps
   %store 413 >> params
# save Spearmans Correlation
   %store 2 >> params
   file='../output/'+fprefix+'_Spearman_'+training_season+'_wk'+str(wk[L])
   %store file >> params

# cross-validated skill maps
   %store 413 >> params
# save 2AFC score
   %store 3 >> params
   file='../output/'+fprefix+'_2AFC_'+training_season+'_wk'+str(wk[L])
   %store file >> params

# cross-validated skill maps
   %store 413 >> params
# save RocBelow score
   %store 10 >> params
   file='../output/'+fprefix+'_RocBelow_'+training_season+'_wk'+str(wk[L])
   %store file >> params

# cross-validated skill maps
   %store 413 >> params
# save RocAbove score
   %store 11 >> params
   file='../output/'+fprefix+'_RocAbove_'+training_season+'_wk'+str(wk[L])
   %store file >> params

#######FORECAST(S)  !!!!!
# Probabilistic (3 categories) maps
   %store 455 >> params
# Output results
   %store 111 >> params
# Forecast probabilities
   %store 501 >> params
   file='../output/'+fprefix+'_CCAFCST_PROB_train_'+training_season+'_'+mon+str(fday)+'_wk'+str(wk[L])
   %store file >> params
#502 # Forecast odds
#Exit submenu
   %store 0 >> params
    
# Deterministic values 
   %store 454 >> params
# Output results
   %store 111 >> params
# Forecast values
   %store 511 >> params
   file='../output/'+fprefix+'_CCAFCST_values_train_'+training_season+'_'+mon+'_'+str(fday)+'_wk'+str(wk[L])
   %store file >> params
#Exit submenu
   %store 0 >> params  
    
# Prediction limits
   %store 454 >> params
# Output results
   %store 111 >> params
# Forecast values
   %store 513 >> params
   file='../output/'+fprefix+'_CCAFCST_PredictionLimits_train_'+training_season+'_'+mon+'_'+str(fday)+'_wk'+str(wk[L])
   %store file >> params
#Exit submenu
   %store 0 >> params      

#0 # Stop saving  (not needed in newest version of CPT)

# Exit
   %store 0 >> params
    
   print('Executing CPT for Week '+str(wk[L])+'...')
   os.system(cptdir+'CPT.x < params > CPT_stout_train_'+training_season+'_'+mon+'_'+str(fday)+'_wk'+str(wk[L])+'.txt')
   

/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/input

 Hindcasts URL: 

 http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/S/-2/1/0/shiftdatashort/%5BS_lag/M%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/-2/1/0/shiftdatashort/%5BS_lag%5Daverage/add/4./div/X/74/92/RANGE/Y/12/32/RANGE/L1/2/9/VALUES/%5BL1%5Ddifferences/S/(Jun-Aug)/VALUES/S/3/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/3001/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv
Done getting hindcast

### Plot forecast maps using grads



In [ ]:
if grads_plot_forecasts:
    %cd $workdir/output
    print('Calling GRADS to plot the forecasts')
# Construct the GRADS program file    
    GradsProgramFile = '/Users/andy/Dropbox/pgm/grads/pl_BiharProbForecsts_body.gs'

# File top
    fh = open('pgm_top.gs', 'w') 
    fh.write("'open PRCP_CCAFCST_PROB_"+mon+"_"+str(fday)+"_wk1.ctl'\n")
    fh.write("'open PRCP_CCAFCST_PROB_"+mon+"_"+str(fday)+"_wk2.ctl'\n")
    fh.write("'open PRCP_CCAFCST_PROB_"+mon+"_"+str(fday)+"_wk34.ctl'\n")
    fh.close() 

# Append 'GradsProgramFile'
    filenames = ['pgm_top.gs', GradsProgramFile]
    with open('pgm.gs', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                    
# Append lines at bottom
    fh = open('pgm.gs', 'a') 
    if rainfall_frequency:
       fh.write("'draw string 3 8.2 CFSv2 Bihar Rainfall Freq Fcst Probabilities, IC: "+mon+str(fday)+"'\n")
    else:
       fh.write("'draw string 3 8.2 CFSv2 Bihar Weekly Rainfall Fcst Probabilities, IC: "+mon+str(fday)+"'\n")
    fh.write("'gxprint "+fprefix+"_CCAFCST_PROB_"+mon+str(fday)+"_Train_"+training_season+".pdf'\n")
    fh.close() 
      
    os.system("grads -blc 'run ./pgm.gs'" )

### Plot skill maps using grads
* 3 leads, select the skill measure: Spearman, 2AFC, RocAbove, RocBelow

In [ ]:
if grads_plot_skill:
    %cd $workdir/output
    print('Calling GRADS to plot skill maps')
# Construct the GRADS program file    
    GradsProgramFile = '/Users/andy/Dropbox/pgm/grads/pl_BiharSkill_body.gs'

# File top
    fh = open('pgm_top.gs', 'w') 
    fh.write("'open "+fprefix+"_"+skill_score+"_"+mon+"_wk1.ctl'\n")
    fh.write("'open "+fprefix+"_"+skill_score+"_"+mon+"_wk2.ctl'\n")
    fh.write("'open "+fprefix+"_"+skill_score+"_"+mon+"_wk34.ctl'\n")
    fh.close() 

# Append 'GradsProgramFile'
    filenames = ['pgm_top.gs', GradsProgramFile]
    with open('pgm.gs', 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                    
# Append lines at bottom
    fh = open('pgm.gs', 'a') 
    if rainfall_frequency:
       fh.write("'draw string 4 9 CFSv2/CCA Bihar Weekly Rainfall Freq Fcst "+skill_score+" Skill "+training_season+"'\n")
    else:
       fh.write("'draw string 4 9 CFSv2/CCA Bihar Weekly Rainfall Fcst "+skill_score+" Skill "+training_season+"'\n")
    fh.write("'gxprint "+fprefix+"_"+skill_score+training_season+".pdf'\n")
    fh.close() 
    
    os.system("grads -bpc 'run ./pgm.gs'" )

In [ ]:
%pwd

Done!
----